In [24]:
%load_ext autoreload
%autoreload 2

# Capturing a video to a file with the Pi

In [45]:
%%writefile 2017-05-17_capture_videos_with_openretina.py
#! /usr/bin/env python3
# -*- coding: utf8 -*-

import picamera

class Rec_video():
    def __init__(self):
        cam = picamera.PiCamera()
        cam.resolution = (640,480) #Defining the camera resolution        
        cam.start_recording(output='video_picam2', format='rgb') #Start recording in 'filename.filetype'
        cam.wait_recording(5) #Better integrated time.wait() that allow to continually check for errors
        cam.stop_recording() #Stop recording

        print('Recording end') #Control line
        
rec = Rec_video() #Calling the Rec_video class

Overwriting 2017-05-17_capture_videos_with_openretina.py


In [46]:
!git add 2017-05-17_capture_videos_with_openretina.py
!git commit -m' updating *_capture_videos_with_openRetina.py '; git push
!ssh pi@10.164.7.213 "cd ~/openRetina ; git pull; cd ~/INT-internship ; git pull"

[master b3dfa31]  updating *_capture_videos_with_openRetina.py
 1 file changed, 1 insertion(+), 1 deletion(-)
Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 337 bytes | 0 bytes/s, done.
Total 3 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:pierrealbiges/INT-internship.git
   2db1afa..b3dfa31  master -> master
Already up-to-date.
From https://github.com/pierrealbiges/INT-internship
   2db1afa..b3dfa31  master     -> origin/master
Updating 2db1afa..b3dfa31
Fast-forward
 2017-05-17_capture_videos_with_openretina.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [47]:
!ssh pi@10.164.7.213 "cd ~/INT-internship ; python3 2017-05-17_capture_videos_with_openretina.py"

Recording end


# Sending the video file to the computer

In [64]:
%%writefile 2017-05-17_capture_videos_with_openretina_client.py
""" Pi side """

import zmq
CHUNK_SIZE = 250000

try:
    context = zmq.Context()
    router = context.socket(zmq.ROUTER)
    
    router.sndhwm = router.rcvhwm = CHUNK_SIZE
    router.hwm = CHUNK_SIZE
    router.bind("tcp://*:5555")
    print("Connected to computer")
    
    file = open("video_picam","rb")
    print('File opened')
    while True:
        identity, command = router.recv_multipart()
        
        assert command == b"fetch"
        print('Command received')
        
        while True:
            data = file.read(CHUNK_SIZE)
            router.send_multipart([identity, data])
            print('Sending data...')
            if not data:
                print('\nNo more data')
                break
        if not data: break

finally:
    router.close()
    context.term()
    print("\nConnection closed")

Overwriting 2017-05-17_capture_videos_with_openretina_client.py


In [65]:
!git add 2017-05-17_capture_videos_with_openretina_client.py
print('PUSHING')
!git commit -m' updating *_capture_videos_with_openRetina_client.py '; git push
print('\nPULLING')
!ssh pi@10.164.7.213 "cd ~/openRetina ; git pull; cd ~/INT-internship ; git pull"

PUSHING
[master cef7630]  updating *_capture_videos_with_openRetina_client.py
 1 file changed, 1 insertion(+)
Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 353 bytes | 0 bytes/s, done.
Total 3 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:pierrealbiges/INT-internship.git
   88b9d7d..cef7630  master -> master

PULLING
Already up-to-date.
From https://github.com/pierrealbiges/INT-internship
   88b9d7d..cef7630  master     -> origin/master
Updating 88b9d7d..cef7630
Fast-forward
 2017-05-17_capture_videos_with_openretina_client.py | 1 +
 1 file changed, 1 insertion(+)


In [66]:
!ssh pi@10.164.7.213 "cd ~/INT-internship/ ; python3 2017-05-17_capture_videos_with_openretina_client.py"

Connected to computer
File opened
Command received
Sending data...
Sending data...
Sending data...

No more data
Connection closed
